In [1]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7f8f1464e510>
2.0.0+cu117


'%.4f'

In [7]:
lib.utils.set_seed()
train_x, train_y = lib.datasets.to_plain_odd(*lib.datasets.get_mnist_test())
network = lib.livenet.LiveNet(784, None, 2)

In [ ]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x) // 500)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.001
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=1)


In [4]:

trainer.step(100)


Iˈ0.000 1.032 = 1.032+0.000 lib/trainer.py:89
Iˈ0.334 0.894 = 0.894+0.000 lib/trainer.py:89
Iˈ0.658 1.025 = 1.025+0.000 lib/trainer.py:89
Iˈ1.007 0.922 = 0.922+0.000 lib/trainer.py:89
Iˈ1.337 0.911 = 0.911+0.000 lib/trainer.py:89
Iˈ1.637 1.007 = 1.007+0.000 lib/trainer.py:89
Iˈ1.946 0.915 = 0.915+0.000 lib/trainer.py:89
Iˈ2.246 0.875 = 0.875+0.000 lib/trainer.py:89
Iˈ2.575 1.287 = 1.287+0.000 lib/trainer.py:89
Iˈ2.913 1.051 = 1.051+0.000 lib/trainer.py:89
Iˈ3.220 0.627 = 0.627+0.000 lib/trainer.py:89
Iˈ3.593 1.063 = 1.063+0.000 lib/trainer.py:89
Iˈ3.946 0.776 = 0.776+0.000 lib/trainer.py:89
Iˈ4.268 0.576 = 0.576+0.000 lib/trainer.py:89
Iˈ4.617 0.968 = 0.968+0.000 lib/trainer.py:89
Iˈ5.048 0.927 = 0.927+0.000 lib/trainer.py:89
Iˈ5.506 0.787 = 0.787+0.000 lib/trainer.py:89
Iˈ5.878 0.785 = 0.785+0.000 lib/trainer.py:89
Iˈ5.883 S45->D0 died at tick 17 with k=0.001 lib/livenet.py:191
Iˈ5.883 S48->D0 died at tick 17 with k=0.000 lib/livenet.py:191
Iˈ5.896 S194->D0 died at tick 17 with k=0.00

In [5]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1513

In [6]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [29]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.6f}")

D0 0.000000
D1 0.000000


In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



S1->D1 0.927462
S1->D1 -0.946442
S1->D1 -0.750392
S1->D1 -0.771363
S1->D1 3.772857
